In [1]:
import numpy as np;
import pandas as pd;


## Initialize Parameters

In [2]:
## parameters and global variables

district = "tvm"

zetaPanchayat = 0.1
zetaMunicipality = 0.2
zetaCorporation = 0.3


muForSmall = 0.09
muForMedium = 0.04
muForLarge = 0.02



## Load and Initialize Data

In [3]:
## Load population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Load distance data
distanceData = pd.read_csv("../data/" + district + "_distance_matrix.csv")

## Load area data
areaData = pd.read_csv("../data/" + district + "_area_data.csv")

## Number of regions
r = len(initDataDF.index)

## Sort population data and distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
distanceData.sort_values(by=['fromName', 'toName'], inplace=True)
areaData.sort_values(by=['name'], inplace=True)


## Initialize Job 
initDataDF.loc[initDataDF['type']=='P','J'] = initDataDF[initDataDF['type']=='P']['N']*zetaPanchayat
initDataDF.loc[initDataDF['type']=='M','J'] = initDataDF[initDataDF['type']=='M']['N']*zetaMunicipality
initDataDF.loc[initDataDF['type']=='C','J'] = initDataDF[initDataDF['type']=='C']['N']*zetaCorporation

## Initialize T
areaData['areaSqKm'] = areaData['area']/(1000*1000)

initDataDF.loc[areaData['areaSqKm'] <= 25,'T'] = initDataDF[areaData['areaSqKm'] <= 25]['N']*muForSmall
initDataDF.loc[(areaData['areaSqKm'] > 25) & (areaData['areaSqKm'] <= 100),'T'] = initDataDF[(areaData['areaSqKm'] > 25) & (areaData['areaSqKm'] <= 100)]['N']*muForMedium
initDataDF.loc[areaData['areaSqKm'] > 100,'T'] = initDataDF[areaData['areaSqKm'] > 100]['N']*muForLarge

initDataDF.tail(15)

,name,type,S,E,I,H,R,N,J,T
62,Thirupuram__Thiruvananthapuram,P,18898,0,0,0,0,18898,1889.8,1700.82
63,Thiruvananthapuram(C)__Thiruvananthapuram,C,966856,0,0,0,0,966856,290056.8,19337.12
64,Tholikkodu__Thiruvananthapuram,P,25274,0,0,0,0,25274,2527.4,1010.96
65,Uzhamalakkal__Thiruvananthapuram,P,21472,0,0,0,0,21472,2147.2,1932.48
66,Vakkam__Thiruvananthapuram,P,16533,0,0,0,0,16533,1653.3,1487.97
67,Vamanpuram__Thiruvananthapuram,P,21038,0,0,0,0,21038,2103.8,1893.42
68,Varkala(M)__Thiruvananthapuram,M,40048,0,0,0,0,40048,8009.6,3604.32
69,Vellanad__Thiruvananthapuram,P,31156,0,0,0,0,31156,3115.6,2804.04
70,Vellarada__Thiruvananthapuram,P,40206,0,0,0,0,40206,4020.6,1608.24
71,Vembayam__Thiruvananthapuram,P,38630,0,0,0,0,38630,3863.0,1545.20


## Matrix computation of TMatrix

### Prepare Distance matrix

In [4]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)
np.fill_diagonal(distanceMatrix,1)
distanceMatrix


array([[    1, 25049, 52074, ..., 17064, 19018, 19149],
       [25049,     1, 28368, ..., 12943, 16264, 14846],
       [52074, 28368,     1, ..., 35843, 36125, 42371],
       ...,
       [17064, 12943, 35843, ...,     1,  4233, 19475],
       [19018, 16264, 36125, ...,  4233,     1, 23689],
       [19149, 14846, 42371, ..., 19475, 23689,     1]])

### Job matrix

In [5]:
jobMatrix = np.tile(initDataDF['J'].to_numpy(),(r,1))
jobMatrix


array([[1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       ...,
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9]])

### Job by Distance Square matrix

In [6]:
tPropJByDSqMatrix = jobMatrix/np.multiply(distanceMatrix,distanceMatrix)
np.fill_diagonal(tPropJByDSqMatrix,0)
tPropJByDSqMatrix


array([[0.00000000e+00, 5.05010419e-06, 6.41516043e-07, ...,
        1.24362896e-05, 8.78141459e-06, 7.15847586e-06],
       [2.53724426e-06, 0.00000000e+00, 2.16168278e-06, ...,
        2.16163622e-05, 1.20071362e-05, 1.19095088e-05],
       [5.87085273e-07, 3.93752829e-06, 0.00000000e+00, ...,
        2.81866724e-06, 2.43376396e-06, 1.46209414e-06],
       ...,
       [5.46740667e-06, 1.89152124e-05, 1.35406869e-06, ...,
        0.00000000e+00, 1.77254647e-04, 6.92082440e-06],
       [4.40162842e-06, 1.19791607e-05, 1.33301086e-06, ...,
        2.02095189e-04, 0.00000000e+00, 4.67755935e-06],
       [4.34161056e-06, 1.43767993e-05, 9.68973659e-07, ...,
        9.54767394e-06, 5.65979514e-06, 0.00000000e+00]])

### Sum of Job by Distance Square matrix

In [7]:
tPropMatrixSum = np.tile(tPropJByDSqMatrix.sum(axis=1),(r,1)).T
tPropMatrixSum

array([[0.00122143, 0.00122143, 0.00122143, ..., 0.00122143, 0.00122143,
        0.00122143],
       [0.00346774, 0.00346774, 0.00346774, ..., 0.00346774, 0.00346774,
        0.00346774],
       [0.00271192, 0.00271192, 0.00271192, ..., 0.00271192, 0.00271192,
        0.00271192],
       ...,
       [0.00432973, 0.00432973, 0.00432973, ..., 0.00432973, 0.00432973,
        0.00432973],
       [0.00586332, 0.00586332, 0.00586332, ..., 0.00586332, 0.00586332,
        0.00586332],
       [0.00087303, 0.00087303, 0.00087303, ..., 0.00087303, 0.00087303,
        0.00087303]])

### Ti propotion matrix

In [8]:
tPropMatrix = tPropJByDSqMatrix/tPropMatrixSum
tPropMatrix

array([[0.        , 0.00413459, 0.00052522, ..., 0.01018176, 0.00718946,
        0.00586074],
       [0.00073167, 0.        , 0.00062337, ..., 0.00623357, 0.00346253,
        0.00343438],
       [0.00021648, 0.00145193, 0.        , ..., 0.00103936, 0.00089743,
        0.00053914],
       ...,
       [0.00126276, 0.00436869, 0.00031274, ..., 0.        , 0.040939  ,
        0.00159844],
       [0.00075071, 0.00204307, 0.00022735, ..., 0.03446773, 0.        ,
        0.00079777],
       [0.00497305, 0.01646773, 0.0011099 , ..., 0.01093627, 0.00648295,
        0.        ]])

### Ti matrix 
`number of people traveling out of region-i`

In [9]:
TiMatrix = np.tile(initDataDF['T'].to_numpy(),(r,1)).T
TiMatrix

array([[ 636.8 ,  636.8 ,  636.8 , ...,  636.8 ,  636.8 ,  636.8 ],
       [2851.83, 2851.83, 2851.83, ..., 2851.83, 2851.83, 2851.83],
       [1565.64, 1565.64, 1565.64, ..., 1565.64, 1565.64, 1565.64],
       ...,
       [3259.08, 3259.08, 3259.08, ..., 3259.08, 3259.08, 3259.08],
       [2858.49, 2858.49, 2858.49, ..., 2858.49, 2858.49, 2858.49],
       [ 524.98,  524.98,  524.98, ...,  524.98,  524.98,  524.98]])

### Tij Matrix

In [10]:
TijMatrix = np.multiply(TiMatrix, tPropMatrix)
TijMatrix

array([[  0.        ,   2.63290663,   0.33445881, ...,   6.4837453 ,
          4.57825102,   3.73212073],
       [  2.08660316,   0.        ,   1.77774533, ...,  17.7770704 ,
          9.87454332,   9.7942556 ],
       [  0.33893448,   2.27320314,   0.        , ...,   1.62726531,
          1.405054  ,   0.84409222],
       ...,
       [  4.11543701,  14.23789554,   1.01923722, ...,   0.        ,
        133.42346395,   5.20945644],
       [  2.14588667,   5.84009347,   0.6498709 , ...,  98.52566626,
          0.        ,   2.28040882],
       [  2.61074987,   8.64523118,   0.58267498, ...,   5.74132298,
          3.40341659,   0.        ]])

### Add Ni - Tij to diagonal elements

In [11]:
row,col = np.diag_indices_from(TijMatrix)
TijMatrix[row,col] = initDataDF['N'].to_numpy() - TijMatrix.sum(axis=1)
TijMatrix

array([[1.52832000e+04, 2.63290663e+00, 3.34458811e-01, ...,
        6.48374530e+00, 4.57825102e+00, 3.73212073e+00],
       [2.08660316e+00, 2.88351700e+04, 1.77774533e+00, ...,
        1.77770704e+01, 9.87454332e+00, 9.79425560e+00],
       [3.38934475e-01, 2.27320314e+00, 1.58303600e+04, ...,
        1.62726531e+00, 1.40505400e+00, 8.44092216e-01],
       ...,
       [4.11543701e+00, 1.42378955e+01, 1.01923722e+00, ...,
        3.29529200e+04, 1.33423464e+02, 5.20945644e+00],
       [2.14588667e+00, 5.84009347e+00, 6.49870904e-01, ...,
        9.85256663e+01, 2.89025100e+04, 2.28040882e+00],
       [2.61074987e+00, 8.64523118e+00, 5.82674981e-01, ...,
        5.74132298e+00, 3.40341659e+00, 2.57240200e+04]])

## Save output

In [12]:

data = []

for i in range(0,r):
    for j in range(0,r):
        Tij = TijMatrix[i][j]
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);

resultsDF.to_csv("../output/" + district + "_tmatrix_results.csv")


## Old Version
### Function definition

def getTij(i,j,distanceMatrix, initDataDF, tPropMatrix):
    Ti = initDataDF.iloc[i]['T']

    if(i==j):
        return 0;
    
    Tjprop = tPropMatrix[i][j]
    TjpropSum = tPropMatrix[i].sum();
    ## TODO move this tPropMatrix computation.
    return Ti*Tjprop/TjpropSum;
    
    

### Compute TMatrix


data = []

for i in range(0,r):
    TijSum = 0
    for j in range(0,r):
        Tij = getTij(i,j,distanceMatrix, initDataDF, tPropMatrix)
        TijSum += Tij
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);

